In [1]:
#se importan las librerias
import cv2
import matplotlib.pyplot as plt
import numpy as np 
from win32api import GetSystemMetrics
%matplotlib qt  
nm=1e-9
um=1e-6
mm=1e-3
cm=1e-2

#longitud de onda
wl=633*nm
#Tamaño del sensor 6.513mm
TamañoSensor=6.513*mm

# se lee la imagen y se obtienen los parámetros asociados
hologram=cv2.imread('Hologram.tiff',0)
M,N=np.shape(hologram)
hologram=np.pad(hologram, [(int(M/2), int(M/2)), (int(M/2), int(M/2))], mode='constant', constant_values=0)
M,N=np.shape(hologram)
#Calculamos el TamañoPixel, (imagen y sensor cuadrados)
deltax=TamañoSensor/N
deltay=TamañoSensor/M
#Eliminamos el orden cero
#MeanHologram=hologram-hologram.mean()
MeanHologram=hologram


FFT_Hologram=np.fft.fft2(MeanHologram)
FFT_Hologram=np.fft.fftshift(FFT_Hologram)

#Creo la mascara circular centrada en el orden 1
mask2=np.zeros((M,N),dtype="uint8")

#cv2.circle(mask2,(3450,1160),2040,1,-1)
cv2.circle(mask2,(650,2900),500,1,-1)
#cv2.circle(mask2,(1024,1024),100,1,-1)
FFT_Hologram=FFT_Hologram*mask2

OrdenUno=np.fft.ifftshift(FFT_Hologram)
OrdenUno=np.fft.ifft2(OrdenUno)
MeanHologram=np.abs(OrdenUno)

plt.imshow(np.log(np.abs(FFT_Hologram)+0.0001))

In [61]:
#Metodo para reescalar las imagenes y verlas con cv2
def get_resized_for_display_img(img):
    screen_w, screen_h = GetSystemMetrics(0), GetSystemMetrics(1)
    print("screen size",screen_w, screen_h)
    h,w,channel_nbr = img.shape
    # img get w of screen and adapt h
    h = h * (screen_w / w)
    w = screen_w
    if h > screen_h: #if img h still too big
        # img get h of screen and adapt w
        w = w * (screen_h / h)
        h = screen_h
    w, h = w*0.9, h*0.9 # because you don't want it to be that big, right ?
    w, h = int(w), int(h) # you need int for the cv2.resize
    return cv2.resize(img, (w, h))

In [62]:
# ESPECTRO ANGULAR POR FFT
def ang_spec(entrada,z, del_x, del_y, wave_length):
    
    k = 2*np.pi/wave_length
    M, N = np.shape(entrada)

    dist = M*del_x**2/wave_length
    
    # Dimensiones longitudinales de la imagen
    Lx = del_x*N
    Ly = del_y*M
    # Tamaño muestral en las frecuencias
    del_fx = 1/Lx
    del_fy = 1/Ly
        
    if N%2==0:
        x = np.arange(-int(N/2),int(N/2),1)
    else:
        x = np.arange(-int(N/2),int(N/2)+1,1)

    if M%2==0:
        y = np.arange(-int(M/2),int(M/2),1)
    else:
        y = np.arange(-int(M/2),int(M/2)+1,1)
        
        
    p, q = np.meshgrid(x,y)
    fx = p*del_fx
    fy = q*del_fy
        
    # espectro angular en el plano de entrada
    A_0 = np.fft.fftshift(np.fft.fft2(entrada)) #np.fft.fft2(entrada) #
    ker = np.exp(1j*z*k*np.emath.sqrt(1 - (wave_length**2)*(fx**2+fy**2) )  )
    ker = ker
    A_z = A_0*ker
    U_z = np.fft.ifft2(A_z)

    print(f'Longitud x de la imagen: {round(del_x*np.shape(entrada)[1]*100, 2)} cm \nLongitud y de la imagen: {round(del_y*np.shape(entrada)[0]*100, 2)} cm ')
    print(f'Máxima distancia de propagación aceptable: {round(dist, 4)} metros o {round(dist*100, 2)} centímetros')

    return U_z

In [69]:
def PlaneWave(M,N,AnglX,dx,dy,wavelength):
  #Centros de la imagen
    Mcen = int(M/2)
    Ncen = int(N/2)
   #Arreglos de x y y
    x=np.arange(-Mcen,Mcen)
    y=np.arange(-Ncen,Ncen)
   #Malla de la matriz
    X, Y = np.meshgrid(x,y)
   #Vector de una onda
    k= 2*np.pi/wavelength
    #Angulos en X y Y
    Ax=np.sin(AnglX)
    E_0=1


    wave =E_0* np.exp(1j*k*(Ax*X*dx))

    return wave
wave_ref=PlaneWave(M,N,0.017,TamañoPixel,TamañoPixel,wl)
wave_ref_2=np.conj(wave_ref)

#Reconstruccion Con espectro angular

image=ang_spec(MeanHologram*wave_ref_2,73*mm,TamañoPixel,TamañoPixel,633*nm)
image=np.abs(image)
#mostrar la imagen
plt.imshow(image,cmap="gray")


Longitud x de la imagen: 0.65 cm 
Longitud y de la imagen: 0.65 cm 
Máxima distancia de propagación aceptable: 0.0164 metros o 1.64 centímetros
